---
title: "Movie-Recommendation Model"
format:
    html: 
        code-fold: true
---

# Code

Code for this webpage can be found [here.](https://github.com/dcorc7/Movie-Recommendation-Model/recommender.ipynb)